**Applied Machine Learning - Homework 4 - Task3**

Amaury Sudrie (UNI: AS5961)
Maxime Tchibozo (UNI: MT3390)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/My Drive/AML/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import torch
#from tqdm import tqdm

In [0]:
df = pd.read_csv('winemag-data-130k-v2.csv')

In [0]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [0]:
df = df.drop(columns="Unnamed: 0")

In [0]:
df[['description','designation','title']] = df[['description','designation','title']].fillna('missing')

## **Question 3.1**
Fine-tune a BERT model on the text data alone using the transformers library. 

Due to the high memory and computational cost of Bert transformer embedding, we will only encode two of the three text features : **description** and **title**, and see if those embeddings are sufficient to obtain a good score with a Linear Regression model.

In [0]:
pip install transformers

In [0]:
pip install category_encoders

In [0]:
import tensorflow as tf
from transformers import BertModel, BertTokenizer, BertConfig, pipeline

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from category_encoders.target_encoder import TargetEncoder


In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

config = BertConfig(
    hidden_size=128, 
    num_hidden_layers=6, 
    num_attention_heads=6).from_pretrained('bert-base-uncased')

In [0]:
X, y = df.drop(columns="points"),df["points"]
X_train, X_test, y_train, y_test = train_test_split(X,y)

#X_train, y_train = X_train.iloc[::3], y_train.iloc[::3]
print(np.shape(X_train))

(97478, 12)


In [0]:
nlp = pipeline(task="feature-extraction",
               model=model,
               tokenizer=tokenizer,
               device=0)

In [0]:
#import time
BERT = []
new_train_vect = []
count = 1

for t in X_train['description']:
  #tx = time.time()
  BERT.append( np.array(nlp(t))[0,0] )
  if count%1000 == 0:
    new_train_vect.append( np.vstack(BERT) )
    del(BERT)
    BERT = []

  count += 1
  #print(time.time()-tx)


new_train_vect.append(BERT)
del(BERT)

In [0]:
new_train = np.vstack(new_train_vect)
new_train.shape

(97478, 768)

We first fit a Linear Regression model using only the **description** component. Bert has embedded the **description** of each row as a vector with 768 dimensions.

In [0]:
model_LR = LinearRegression()

scores = cross_val_score(model_LR, new_train, y_train)
print("Score Linear Regression", np.mean(scores))
#model_LR.fit(new_train, y_train)

Score Linear Regression 0.5960626179210056


Using **description** alone with Bert embedding yields a better score than using **description**,**title** and **designation** together with word2vec embedding (sccore of 56%). We will further improve upon this model by also considering **title**.

In [0]:
#import time
BERT = []
new_train_vect2 = []
count = 1

for t in X_train['title']:
  #tx = time.time()
  BERT.append( np.array(nlp(t))[0,0] )
  if count%1000 == 0:
    new_train_vect2.append( np.vstack(BERT) )
    del(BERT)
    BERT = []

  count += 1
  #print(time.time()-tx)


new_train_vect2.append(BERT)
del(BERT)

In [0]:
new_train2 = np.hstack( [np.vstack(new_train_vect2), new_train] )
new_train2.shape

(97478, 1536)

In [0]:
model_LR = LinearRegression()

scores = cross_val_score(model_LR, new_train2, y_train)
print("Score Linear Regression", np.mean(scores))
#model_LR.fit(new_train, y_train)

Score Linear Regression 0.6429778134483181


A Linear Regression model using Bert embedding for **description** and **title** yields a similar score to a ridge model with Bag of Words embedding using Tf-idf (score was 68%). However, the Bert-embedded vectors have shape (,1536), whereas the Tf-idf vectors had size (,4000). Since Linear Regression and Ridge usually perform better when we increase the number of features (so long as those features are not colinear), this means that the Bert embedded vectors contain much more information on the target than both word2vec vectors and Tf-idf with Bag of Word vectors. In other words, they contain more semantic information than the other methods (i.e information relative to the ordering of the words).

## **Question 3.2**
How does this model compare to a BoW model, and how does it compare to a model using all features?

First let us have a look on the Bag of Word only while considering only the **title** and **description** features.

In [0]:
vect1 = make_pipeline(CountVectorizer(min_df=2, max_features=2000))
vect2 = make_pipeline(CountVectorizer(min_df=2, max_features=1000, max_df=0.95))

BoW_train = sp.sparse.hstack([
    vect1.fit_transform(X_train["description"]),
    vect2.fit_transform(X_train["title"]),
])

LR = LinearRegression()
scores = cross_val_score(LR, BoW_train, y_train)
print("Score Linear Regression", np.mean(scores))

Score Linear Regression 0.7142438999014576


Now let us compare to the complete model with all features

In [0]:
cont = ["price"]
cat_TE = ["province", "region_1", "variety", "winery"]
cat_OHE = ["country", "region_2", "taster_name", "taster_twitter_handle"]

X_train[cat_OHE + cat_TE] = X_train[cat_OHE + cat_TE].fillna('missing')
Cont_Imputer = SimpleImputer(strategy='median')
X_train[cont] = Cont_Imputer.fit_transform(X_train[cont])

tfidf1 = make_pipeline(CountVectorizer(min_df=2, max_features=2000), TfidfTransformer())
tfidf2 = make_pipeline(CountVectorizer(min_df=2, max_features=1000, max_df=0.95), TfidfTransformer())
tfidf3 = make_pipeline(CountVectorizer(min_df=2, max_features=1000, max_df=0.95), TfidfTransformer())
preprocess = make_column_transformer(
    (TargetEncoder(), cat_TE),
    (OneHotEncoder(handle_unknown='ignore'), cat_OHE),
    (StandardScaler(), cont)
)

train_all_feature = sp.sparse.hstack([
    tfidf1.fit_transform(X_train["description"]),
    tfidf2.fit_transform(X_train["designation"]),
    tfidf3.fit_transform(X_train["title"]),
    preprocess.fit_transform(X_train[cont + cat_OHE + cat_TE], y_train),
])

LR = LinearRegression()
scores = cross_val_score(LR, train_all_feature, y_train)
print("Score Linear Regression", np.mean(scores))

Score Linear Regression 0.7781389248802411


The Bag of Word models still yield a better score than our previous Bert model. However, there are some important caveats to take into account. The Bag-of-Words embedding model on the other hand has 4009 dimensions whereas the Bert model has 1536.
Due to memory limits on Colab, we cannot use the third text feature : **designation**, but seeing how well Bert embedding performs without it, we should expect that a model using this feature embedded with Bert should have a comparable (if not better score) than our state of the art model which used Bag of words with Tf-idf (score of ~77%).